In [1]:
import requests # 웹 페이지 소스를 얻기 위한 패키지(기본 내장 패키지이다.)
from bs4 import BeautifulSoup # 웹 페이지 소스를 얻기 위한 패키지, 더 간단히 얻을 수 있다는 장점이 있다고 한다.
from datetime import datetime                                # (!pip install beautifulsoup4 으로 다운받을 수 있다.)
import pandas as pd # 데이터를 처리하기 위한 가장 기본적인 패키지
import time # 사이트를 불러올 때, 작업 지연시간을 지정해주기 위한 패키지이다. (사이트가 늦게 켜지면 에러가 발생하기 때문)
import urllib.request #
from selenium.webdriver import Chrome
import json
import re     
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import datetime as dt

In [2]:
df= pd.read_csv('mango_final.csv')
df

,서칭,이름,주소,전화번호,메뉴/가격,가격대,영업시간,정보,특징,리뷰,평점
0,국수 / 면 요리,진미평양냉면,서울특별시 강남구 학동로 305-3지번서울시 강남구 논현동 115-10,02-515-3469,"냉면 (물/비빔)11,000원편육 (소고기)28,000원제육26,000원제육반13,...",만원-2만원,11:00 - 21:30,주차공간없음,,[' 듣던대로 만두맛집!물냉면이랑 만두 한접시 주문했는데 전 평냉은...,4.6
1,국수 / 면 요리,서관면옥,서울특별시 서초구 서초대로56길 11지번서울시 서초구 서초동 1675-5,02-521-9945,"평양냉면13,000원골동냉면13,000원선비냉면13,000원맛박이냉면15,000원돝...",만원-2만원,11:00 - 22:00,무료주차 가능,,[' 깔끔하게 맛있어요근처 설눈보다 만두는 좀 덜 맛있고나머지는 다...,4.5
2,국수 / 면 요리,을지면옥,서울특별시 중구 충무로 72-1지번서울시 중구 입정동 156,02-2266-7052,"냉면12,000원비빔냉면12,000원소고기국밥10,000원수육28,000원편육24,...",만원-2만원,11:00 - 21:00,주차공간없음,,"[' 내 기준 평양냉면 최고 ', "" ...",4.4
3,고기 요리,봉피양,서울특별시 송파구 양재대로71길 1-4 벽제갈비지번서울시 송파구 방이동 205-8 ...,02-415-5527,"돼지본갈비 (270g)27,000원한우떡갈비 (200g)30,000원평양냉면14,000원",만원-2만원,11:00 - 22:00,발렛,,[' 인생 국밥. 봉피양의 한우양곰탕은 찐입니다... 정말 몸이 보...,4.4
4,국수 / 면 요리,설눈,서울특별시 서초구 서초대로46길 20-7지번서울시 서초구 서초동 1551-9,02-6959-9339,NaN,만원-2만원,11:00 - 21:00,무료주차 가능,,[' 소고기 수육 완전 부들부들비빔도 맛있고 물냉도 맛있어요물김치 ...,4.4
...,...,...,...,...,...,...,...,...,...,...,...
7426,기타 양식,리틀크레이지피자,서울특별시 서초구 신반포로 38지번서울시 서초구 반포동 1048,02-591-0282,NaN,만원 미만,11:00 - 21:00,주차공간없음,,[' 갈릭스페셜피자가 제일 맛나다. 넘나 좋아하는 최애 피자.오븐스...,4.1
7427,카페 / 디저트,박종근과자점,서울특별시 서초구 신반포로 17지번서울시 서초구 반포동 1048,02-599-4295,NaN,만원 미만,월-금: 06:30 - 23:00 토: 06:30 - 22:00 일: 14:00 -...,주차공간없음,,[' 별 4개 ☆☆☆☆2000년부터 동네에 생긴 빵집인데 초창기에는...,3.9
7428,전통 주점 / 포차,스마일포차,서울특별시 서초구 신반포로 49지번서울시 서초구 반포동 938,02-534-0236,NaN,만원-2만원,17:00 - 03:30,무료주차 가능,,[' 와 코로나에도 끄떡없는 이곳. 테이블 간격이 너무 좁아 지금 ...,3.7
7429,카페 / 디저트,에이치플러스와이,서울특별시 서초구 신반포로 45 반포아파트지번서울시 서초구 반포동 809 반포아파트,02-3477-2612,NaN,만원 미만,09:00 - 21:00,무료주차 가능,,[' 예전엔 몰랐는데.. 타르트지가 두꺼운편.부드러운 식감을 찾는 ...,NaN


In [7]:
df['리뷰'][0]
df

'듣던대로 만두맛집물냉면이랑 만두 한접시 주문했는데 전 평냉은 비냉이 취향인 듯해요물냉은 정말 고기 끓인 육수에 소금 찌끔 넣은 담백한 국물에 면발은 그 맛조차 덜 배서 거의 무맛그리고 뭔가 먹고 나니 입가가 기름진 느낌만두는 슴슴한 것도 아니고 짠 것도 아니고 정말 딱 담백하게 맛있어요 많이 먹어도 부담스럽지 않을 것 같아요 재방문하게 된다면 비냉이랑 만두 먹을 거예용 한줄평 의외의 만두 맛집주관적 평점 요새 평양냉면을 즐겨먹기 시작한 평린이의 능라도 우래옥에 이은 번째 평양냉면집일단 분위기는 오래된 맛집 분위기다 칼국수집 같은 분위기에 아이들부터 노인분들까지 남녀노소 모두가 와있다 만두가 맛있다는 남친의 말에 평양냉면 인과 만두한접시 주문일단 밑반찬으로는 김치와 절인무가 나오는데 김치가 맛있어서 다음에는 제육과 먹어보고 싶었다 김치를 먹다보니 메인인 평양냉면 등장 국물은 우래옥보다 연하고 능라도와 비슷한 정도의 육향과 간을 가지고 있었다 뭔가 우래옥이 보신될것같은 진함이라면 여기는 깔끔한 느낌이 강했다 면은 능라도가 끊기고 거친 면이라 좋았는데 우래옥보다는 거친듯하지만 인상깊지 않았다 고명으로 소고기 돼지고기가 하나씩 있었는데 의외로 계란 고명이 베스트였다 잘 삶겨져서 계란 비린맛이 없고 국물과 참 잘 어울렸다냉면을 즐기다보니 만두 한접시개가 나왔다 꽤 큰 만두였는데 접해본 이북만두중 제일 맛있었다 의외의 만두맛집 보통 이북만두는 숙주와 두부의 조합으로 담백한 맛이 일품이지만 나에게는 너무 심심했는데 이곳의 만두는 너무 맛있었다 같이 나오는 소스도 맛있고 육즙도 풍부해서 하정우씨 김먹방찍듯이 그 큰 만두를 입에 잔뜩넣고 먹었다 다음에도 오면 만두는 꼭 시킬듯만족스러운 한끼 식사였다 아쉬웠던점은 면은 양이 너무 많아 국물맛이 연해진다는 것이었다 먹다 중간에 육수를 리필받았는데 그때 간이 딱 좋았다 면 양이 많아 남길 정도니 면양을 줄이고 육수양을 늘려주었으면 좋겠다 개인적으로 면은 능라도처렁 육수는 우래옥과 진미평양냉면 사이의 육수로 하는 평양냉면집이 있다면

In [12]:
import numpy as np

ab = np.array(df['리뷰'].tolist())
ab

array(['듣던대로 만두맛집물냉면이랑 만두 한접시 주문했는데 전 평냉은 비냉이 취향인 듯해요물냉은 정말 고기 끓인 육수에 소금 찌끔 넣은 담백한 국물에 면발은 그 맛조차 덜 배서 거의 무맛그리고 뭔가 먹고 나니 입가가 기름진 느낌만두는 슴슴한 것도 아니고 짠 것도 아니고 정말 딱 담백하게 맛있어요 많이 먹어도 부담스럽지 않을 것 같아요 재방문하게 된다면 비냉이랑 만두 먹을 거예용 한줄평 의외의 만두 맛집주관적 평점 요새 평양냉면을 즐겨먹기 시작한 평린이의 능라도 우래옥에 이은 번째 평양냉면집일단 분위기는 오래된 맛집 분위기다 칼국수집 같은 분위기에 아이들부터 노인분들까지 남녀노소 모두가 와있다 만두가 맛있다는 남친의 말에 평양냉면 인과 만두한접시 주문일단 밑반찬으로는 김치와 절인무가 나오는데 김치가 맛있어서 다음에는 제육과 먹어보고 싶었다 김치를 먹다보니 메인인 평양냉면 등장 국물은 우래옥보다 연하고 능라도와 비슷한 정도의 육향과 간을 가지고 있었다 뭔가 우래옥이 보신될것같은 진함이라면 여기는 깔끔한 느낌이 강했다 면은 능라도가 끊기고 거친 면이라 좋았는데 우래옥보다는 거친듯하지만 인상깊지 않았다 고명으로 소고기 돼지고기가 하나씩 있었는데 의외로 계란 고명이 베스트였다 잘 삶겨져서 계란 비린맛이 없고 국물과 참 잘 어울렸다냉면을 즐기다보니 만두 한접시개가 나왔다 꽤 큰 만두였는데 접해본 이북만두중 제일 맛있었다 의외의 만두맛집 보통 이북만두는 숙주와 두부의 조합으로 담백한 맛이 일품이지만 나에게는 너무 심심했는데 이곳의 만두는 너무 맛있었다 같이 나오는 소스도 맛있고 육즙도 풍부해서 하정우씨 김먹방찍듯이 그 큰 만두를 입에 잔뜩넣고 먹었다 다음에도 오면 만두는 꼭 시킬듯만족스러운 한끼 식사였다 아쉬웠던점은 면은 양이 너무 많아 국물맛이 연해진다는 것이었다 먹다 중간에 육수를 리필받았는데 그때 간이 딱 좋았다 면 양이 많아 남길 정도니 면양을 줄이고 육수양을 늘려주었으면 좋겠다 개인적으로 면은 능라도처렁 육수는 우래옥과 진미평양냉면 사이의 육수로 하는 평양냉

In [ ]:
from hanspell import spell_checker

sent = 
spelled_sent = spell_checker.check(sent)

hanspell_sent = spelled_sent.checked

print(sent)
print('-'*50)
print(hanspell_sent)

In [13]:
a = df['리뷰'][0:2]
review = a.tolist()
review


['듣던대로 만두맛집물냉면이랑 만두 한접시 주문했는데 전 평냉은 비냉이 취향인 듯해요물냉은 정말 고기 끓인 육수에 소금 찌끔 넣은 담백한 국물에 면발은 그 맛조차 덜 배서 거의 무맛그리고 뭔가 먹고 나니 입가가 기름진 느낌만두는 슴슴한 것도 아니고 짠 것도 아니고 정말 딱 담백하게 맛있어요 많이 먹어도 부담스럽지 않을 것 같아요 재방문하게 된다면 비냉이랑 만두 먹을 거예용 한줄평 의외의 만두 맛집주관적 평점 요새 평양냉면을 즐겨먹기 시작한 평린이의 능라도 우래옥에 이은 번째 평양냉면집일단 분위기는 오래된 맛집 분위기다 칼국수집 같은 분위기에 아이들부터 노인분들까지 남녀노소 모두가 와있다 만두가 맛있다는 남친의 말에 평양냉면 인과 만두한접시 주문일단 밑반찬으로는 김치와 절인무가 나오는데 김치가 맛있어서 다음에는 제육과 먹어보고 싶었다 김치를 먹다보니 메인인 평양냉면 등장 국물은 우래옥보다 연하고 능라도와 비슷한 정도의 육향과 간을 가지고 있었다 뭔가 우래옥이 보신될것같은 진함이라면 여기는 깔끔한 느낌이 강했다 면은 능라도가 끊기고 거친 면이라 좋았는데 우래옥보다는 거친듯하지만 인상깊지 않았다 고명으로 소고기 돼지고기가 하나씩 있었는데 의외로 계란 고명이 베스트였다 잘 삶겨져서 계란 비린맛이 없고 국물과 참 잘 어울렸다냉면을 즐기다보니 만두 한접시개가 나왔다 꽤 큰 만두였는데 접해본 이북만두중 제일 맛있었다 의외의 만두맛집 보통 이북만두는 숙주와 두부의 조합으로 담백한 맛이 일품이지만 나에게는 너무 심심했는데 이곳의 만두는 너무 맛있었다 같이 나오는 소스도 맛있고 육즙도 풍부해서 하정우씨 김먹방찍듯이 그 큰 만두를 입에 잔뜩넣고 먹었다 다음에도 오면 만두는 꼭 시킬듯만족스러운 한끼 식사였다 아쉬웠던점은 면은 양이 너무 많아 국물맛이 연해진다는 것이었다 먹다 중간에 육수를 리필받았는데 그때 간이 딱 좋았다 면 양이 많아 남길 정도니 면양을 줄이고 육수양을 늘려주었으면 좋겠다 개인적으로 면은 능라도처렁 육수는 우래옥과 진미평양냉면 사이의 육수로 하는 평양냉면집이 있다

In [3]:
df['리뷰']=df['리뷰'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
df['리뷰']=df['리뷰'].str.replace('  ','')
df['리뷰']=df['리뷰'].str.replace('ㅋ','')
df['리뷰']=df['리뷰'].str.replace('ㅎ','')
df['리뷰']=df['리뷰'].str.replace('ㅜ','')
df['리뷰']=df['리뷰'].str.replace('ㅠ','')
df['리뷰']

0       듣던대로 만두맛집물냉면이랑 만두 한접시 주문했는데 전 평냉은 비냉이 취향인 듯해요물...
1       깔끔하게 맛있어요근처 설눈보다 만두는 좀 덜 맛있고나머지는 다 여기가 더 맛있어요 ...
2       내 기준 평양냉면 최고 호선 타고 집가다가 갑자기 평양냉면이 땡겨서 을지로 가에서 ...
3       인생 국밥 봉피양의 한우양곰탕은 찐입니다 정말 몸이 보양되는기분 안에 내용물도 정말...
4       소고기 수육 완전 부들부들비빔도 맛있고 물냉도 맛있어요물김치 김치 다 맛나요 설눈 ...
                              ...                        
7426    갈릭스페셜피자가 제일 맛나다 넘나 좋아하는 최애 피자오븐스파게티는 무난한 맛인데 가...
7427    별 개 년부터 동네에 생긴 빵집인데 초창기에는 그저 그랬던 기억이 있어서 그후로 안...
7428    와 코로나에도 끄떡없는 이곳 테이블 간격이 너무 좁아 지금 이시국에 추천하는 모임장...
7429    예전엔 몰랐는데 타르트지가 두꺼운편부드러운 식감을 찾는 사람들에게 타르트 보단 치케...
7430    삼겹살 가격은 다른 식당의 가격과 비슷했지만 고기에서 나는 잡내는 심했다 그리고 삼...
Name: 리뷰, Length: 7431, dtype: object

In [4]:
df['리뷰'][0]
df

,서칭,이름,주소,전화번호,메뉴/가격,가격대,영업시간,정보,특징,리뷰,평점
0,국수 / 면 요리,진미평양냉면,서울특별시 강남구 학동로 305-3지번서울시 강남구 논현동 115-10,02-515-3469,"냉면 (물/비빔)11,000원편육 (소고기)28,000원제육26,000원제육반13,...",만원-2만원,11:00 - 21:30,주차공간없음,,듣던대로 만두맛집물냉면이랑 만두 한접시 주문했는데 전 평냉은 비냉이 취향인 듯해요물...,4.6
1,국수 / 면 요리,서관면옥,서울특별시 서초구 서초대로56길 11지번서울시 서초구 서초동 1675-5,02-521-9945,"평양냉면13,000원골동냉면13,000원선비냉면13,000원맛박이냉면15,000원돝...",만원-2만원,11:00 - 22:00,무료주차 가능,,깔끔하게 맛있어요근처 설눈보다 만두는 좀 덜 맛있고나머지는 다 여기가 더 맛있어요 ...,4.5
2,국수 / 면 요리,을지면옥,서울특별시 중구 충무로 72-1지번서울시 중구 입정동 156,02-2266-7052,"냉면12,000원비빔냉면12,000원소고기국밥10,000원수육28,000원편육24,...",만원-2만원,11:00 - 21:00,주차공간없음,,내 기준 평양냉면 최고 호선 타고 집가다가 갑자기 평양냉면이 땡겨서 을지로 가에서 ...,4.4
3,고기 요리,봉피양,서울특별시 송파구 양재대로71길 1-4 벽제갈비지번서울시 송파구 방이동 205-8 ...,02-415-5527,"돼지본갈비 (270g)27,000원한우떡갈비 (200g)30,000원평양냉면14,000원",만원-2만원,11:00 - 22:00,발렛,,인생 국밥 봉피양의 한우양곰탕은 찐입니다 정말 몸이 보양되는기분 안에 내용물도 정말...,4.4
4,국수 / 면 요리,설눈,서울특별시 서초구 서초대로46길 20-7지번서울시 서초구 서초동 1551-9,02-6959-9339,NaN,만원-2만원,11:00 - 21:00,무료주차 가능,,소고기 수육 완전 부들부들비빔도 맛있고 물냉도 맛있어요물김치 김치 다 맛나요 설눈 ...,4.4
...,...,...,...,...,...,...,...,...,...,...,...
7426,기타 양식,리틀크레이지피자,서울특별시 서초구 신반포로 38지번서울시 서초구 반포동 1048,02-591-0282,NaN,만원 미만,11:00 - 21:00,주차공간없음,,갈릭스페셜피자가 제일 맛나다 넘나 좋아하는 최애 피자오븐스파게티는 무난한 맛인데 가...,4.1
7427,카페 / 디저트,박종근과자점,서울특별시 서초구 신반포로 17지번서울시 서초구 반포동 1048,02-599-4295,NaN,만원 미만,월-금: 06:30 - 23:00 토: 06:30 - 22:00 일: 14:00 -...,주차공간없음,,별 개 년부터 동네에 생긴 빵집인데 초창기에는 그저 그랬던 기억이 있어서 그후로 안...,3.9
7428,전통 주점 / 포차,스마일포차,서울특별시 서초구 신반포로 49지번서울시 서초구 반포동 938,02-534-0236,NaN,만원-2만원,17:00 - 03:30,무료주차 가능,,와 코로나에도 끄떡없는 이곳 테이블 간격이 너무 좁아 지금 이시국에 추천하는 모임장...,3.7
7429,카페 / 디저트,에이치플러스와이,서울특별시 서초구 신반포로 45 반포아파트지번서울시 서초구 반포동 809 반포아파트,02-3477-2612,NaN,만원 미만,09:00 - 21:00,무료주차 가능,,예전엔 몰랐는데 타르트지가 두꺼운편부드러운 식감을 찾는 사람들에게 타르트 보단 치케...,NaN


In [14]:
a = df['리뷰'][0:2]
review = a.tolist()
review

['듣던대로 만두맛집물냉면이랑 만두 한접시 주문했는데 전 평냉은 비냉이 취향인 듯해요물냉은 정말 고기 끓인 육수에 소금 찌끔 넣은 담백한 국물에 면발은 그 맛조차 덜 배서 거의 무맛그리고 뭔가 먹고 나니 입가가 기름진 느낌만두는 슴슴한 것도 아니고 짠 것도 아니고 정말 딱 담백하게 맛있어요 많이 먹어도 부담스럽지 않을 것 같아요 재방문하게 된다면 비냉이랑 만두 먹을 거예용 한줄평 의외의 만두 맛집주관적 평점 요새 평양냉면을 즐겨먹기 시작한 평린이의 능라도 우래옥에 이은 번째 평양냉면집일단 분위기는 오래된 맛집 분위기다 칼국수집 같은 분위기에 아이들부터 노인분들까지 남녀노소 모두가 와있다 만두가 맛있다는 남친의 말에 평양냉면 인과 만두한접시 주문일단 밑반찬으로는 김치와 절인무가 나오는데 김치가 맛있어서 다음에는 제육과 먹어보고 싶었다 김치를 먹다보니 메인인 평양냉면 등장 국물은 우래옥보다 연하고 능라도와 비슷한 정도의 육향과 간을 가지고 있었다 뭔가 우래옥이 보신될것같은 진함이라면 여기는 깔끔한 느낌이 강했다 면은 능라도가 끊기고 거친 면이라 좋았는데 우래옥보다는 거친듯하지만 인상깊지 않았다 고명으로 소고기 돼지고기가 하나씩 있었는데 의외로 계란 고명이 베스트였다 잘 삶겨져서 계란 비린맛이 없고 국물과 참 잘 어울렸다냉면을 즐기다보니 만두 한접시개가 나왔다 꽤 큰 만두였는데 접해본 이북만두중 제일 맛있었다 의외의 만두맛집 보통 이북만두는 숙주와 두부의 조합으로 담백한 맛이 일품이지만 나에게는 너무 심심했는데 이곳의 만두는 너무 맛있었다 같이 나오는 소스도 맛있고 육즙도 풍부해서 하정우씨 김먹방찍듯이 그 큰 만두를 입에 잔뜩넣고 먹었다 다음에도 오면 만두는 꼭 시킬듯만족스러운 한끼 식사였다 아쉬웠던점은 면은 양이 너무 많아 국물맛이 연해진다는 것이었다 먹다 중간에 육수를 리필받았는데 그때 간이 딱 좋았다 면 양이 많아 남길 정도니 면양을 줄이고 육수양을 늘려주었으면 좋겠다 개인적으로 면은 능라도처렁 육수는 우래옥과 진미평양냉면 사이의 육수로 하는 평양냉면집이 있다

In [15]:
from pykospacing import spacing

spacing_data = []
for sent in a:
    sent1 = spacing(sent)
    spacing_data.append(sent1)
    print(sent1)

듣던 대로 만두 맛집 물냉면이랑 만두 한 접시 주문했는데 전 평냉은 비냉이 취향인 듯해요 물냉은 정말 고기 끓인 육수에 소금 찌끔 넣은 담백한 국물에 면발은 그 맛조차 덜 배서 거의 무맛 그리고 뭔가 먹고 나니 입가가 기름진 느낌만 두는 슴슴한 것도 아니고 짠 것도 아니고 정말 딱 담백하게 맛있어요 많이 먹어도 부담스럽지 않을 것 같아 요 재방문하게 된다면 비냉이랑  만두 먹을 거예용 한 줄 평 의 외의 만두 맛집 주관적 평점 요새 평양냉면을 즐겨 먹기 시작한 평린이의 능라도 우래옥에 이은 번째 평양냉면 집 일단 분위기는 오래된 맛집 분위기 다 칼국수집 같은 분위기에 아이들부터 노인 분들까지 남녀노소 모두가 와 있다 만두가 맛있다는 남친의 말에 평양냉면 인과 만두한 접시 주문일단 밑반찬으로는 김치와 절인 무가 나오는데 김치가 맛있어서 다 음에는 제육과 먹어보고 싶었다 김치를 먹다 보니 메인인 평양냉면 등장 국물은 우래 옥보다 연하고 능라도와 비슷한 정도의 육향과 간을 가지고 있었다 뭔가 우래 옥이 보신될 것 같은 진함이라면 여기는 깔끔한 느낌이 강했다 면은 능라도가 끊기고 거친 면이라 좋았는데 우래 옥보다는 거친 듯하지만 인상 깊지 않았다 고명으로 소고기 돼지고기가 하나씩 있었는데 의 외로 계란 고명이  베스트였다 잘 삶겨져서 계란 비린 맛이 없고 국물과 참 잘 어울렸다 냉면을 즐기다 보니 만두 한 접시개가 나왔다 꽤 큰 만두였는데 접해 본 이북만두 중 제일 맛있었다 의 외의 만두 맛집 보통 이 북만두는 숙주와 두부의 조합으로 담백한 맛이 일품이지만 나에게는 너무 심심했는데 이곳의 만두는 너무 맛있었다 같이 나오는 소스도 맛있고 육즙도 풍부해서 하정우씨 김 먹방찍듯이  그 큰 만두를 입에 잔뜩 넣고 먹었다 다음에도 오면 만두는 꼭 시킬 듯 만족스러운 한 끼 식사였다 아쉬웠던 점은 면은 양이 너무 많아 국물 맛이 연해진다는 것이었다 먹다 중간에 육수를 리필 받았는데 그때 간이 딱 좋았다 면 양이 많아 남길 정도니 면양을 줄이고 육수양을 늘려주었으면 좋겠다 개인적

In [18]:
with open('불용어리스트_gy.txt', "r", encoding='UTF-8') as file:
    strings = file.readlines()
    #print(strings)

In [19]:
불용어=pd.DataFrame({'불용어':strings})
불용어=불용어['불용어'].str.replace('\n','')
stopwords=불용어.to_list()

In [27]:
import urllib.request
from konlpy.tag import Okt
import pandas as pd
import matplotlib.pyplot as plt

okt = Okt()
tokenized_data = []
for sentence in spacing_data:
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    tokenized_data.append(temp_X)
    print('불용어=>',temp_X)

불용어=> ['듣다', '대로', '만두', '맛집', '물냉면', '이랑', '만두', '하다', '접시', '주문', '하다', '전', '평냉', '비', '냉이', '취향', '인', '듯', '하다', '물냉', '정말', '고기', '끓이다', '육수', '소금', '찌', '끄다', '넣다', '담백하다', '국물', '면발', '맛', '덜', '배서', '무맛', '뭔가', '먹다', '나니', '입가', '기르다', '느낌', '만', '두다', '슴슴', '한', '도', '아니다', '짜다', '도', '아니다', '정말', '담백하다', '맛있다', '많이', '먹다', '부담', '스럽지', '않다', '요', '재', '방문', '게', '되다', '비', '냉이', '랑', '만두', '먹다', '거', '용', '하다', '줄', '평', '외', '만두', '맛집', '주관', '적', '평점', '요새', '평양', '냉면', '즐기다', '먹기', '시작', '한', '평린', '이의', '능라도', '우', '래', '옥', '번째', '평양', '냉면', '집', '일단', '분위기', '오래되다', '맛집', '분위기', '다', '칼국수', '집', '분위기', '아이', '노인', '분', '남녀', '노소', '오다', '있다', '만두', '맛있다', '남친', '말', '평양', '냉면', '인과', '만두', '한', '접시', '주문', '일단', '밑반찬', '으로는', '김치', '절', '인', '무가', '나오다', '김치', '맛있다', '다', '음', '에는', '제육', '먹다', '보고', '싶다', '김치', '먹다', '보다', '메인', '인', '평양', '냉면', '등장', '국물', '우', '래', '옥', '보다', '연하다', '능라도', '정도', '육', '향', '간', '가지', '고', '있다', '뭔가', '우', '래', '옥', '보신', '되다', '진하다',

In [22]:
len(tokenized_data)

2

In [28]:
import numpy as np
words = np.hstack(tokenized_data)

In [29]:
from collections import Counter

words_counter = Counter(words)
words_counter = words_counter.most_common()

In [30]:
words_counter_pos = []
for item in words_counter:
  pos_tuple = (item[0], item[1], okt.pos(item[0]))
  words_counter_pos.append(pos_tuple)

In [34]:
words_counter_pos[:10]

[('하다', 25, [('하다', 'Verb')]),
 ('먹다', 19, [('먹다', 'Verb')]),
 ('만두', 18, [('만두', 'Noun')]),
 ('냉면', 17, [('냉면', 'Noun')]),
 ('맛있다', 16, [('맛있다', 'Adjective')]),
 ('맛', 13, [('맛', 'Noun')]),
 ('있다', 12, [('있다', 'Adjective')]),
 ('육수', 11, [('육수', 'Noun')]),
 ('도', 11, [('도', 'Noun')]),
 ('평양', 11, [('평양', 'Noun')])]

In [35]:
wcp_df = pd.DataFrame(words_counter_pos)
wcp_df.columns = ['word', 'count', 'pos']

In [36]:
wcp_df.head(30)

,word,count,pos
0,하다,25,"[(하다, Verb)]"
1,먹다,19,"[(먹다, Verb)]"
2,만두,18,"[(만두, Noun)]"
3,냉면,17,"[(냉면, Noun)]"
4,맛있다,16,"[(맛있다, Adjective)]"
5,맛,13,"[(맛, Noun)]"
6,있다,12,"[(있다, Adjective)]"
7,육수,11,"[(육수, Noun)]"
8,도,11,"[(도, Noun)]"
9,평양,11,"[(평양, Noun)]"


In [ ]:
wcp_df[wcp_df['pos'].map(len) >= 2]